In [1]:
#########CMSC-516########
#########NLP Project: SemEvalTask 2022########
#########To identify Sarcasm on provided tweets by Silviu & Team#########
##########Silviu is the organiser for this task iSarcasm#########
#########Credits: Huggingface & Vincent###########
##########Github Link for both Train and Test data are below#########
##########https://github.com/silviu-oprea/iSarcasm###########
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

     |████████████████████████████████| 3.3 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 3.3 MB 36.2 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 61 kB 459 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
######We are reading the training data#####
data = pd.read_csv("train.En.csv")

In [3]:
######We are using BERT pretrained model(uncased) and takenizer#####
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
#####80:20 split#####
#####preprocessing the training set#######
#####input data with same len to the model with max 512########
X = list(data["tweet"])
y = list(data["sarcastic"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
print("Total Length of the Train.csv File received:",len(X))
print("Sarcastic Tweets out of 3467 is:",data.sarcastic.sum())
print("Length of the Training Data:",len(X_train))
print("Length of the Validation Data:",len(X_val))
#y_train

Total Length of the Train.csv File received: 3467
Sarcastic Tweets out of 3467 is: 867
Length of the Training Data: 2773
Length of the Validation Data: 694


In [5]:
#########We now create the dataset in torch class as per the trainer API#########
##########To reuse the model on test data prediction we make labels=none######
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
#######Now creating batches of data########
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item
###########length###########
    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [6]:
##########Defining our Evaluation metrics#######
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [7]:
#############We are defining the hyperparameters #########
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    seed=0,
    load_best_model_at_end=True,
)
##########Defining the trainer function########
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [8]:
#########Training the trainer model#########
#########Fine tuning BERT model##############
trainer.train()

***** Running training *****
  Num examples = 2773
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2082


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.561700,0.569431,0.727666,0.407407,0.260355,0.317690
1000,0.377300,1.112669,0.730548,0.419643,0.278107,0.334520
1500,0.108800,1.550125,0.750720,0.481818,0.313609,0.379928
2000,0.019600,1.935739,0.742075,0.463768,0.378698,0.416938


***** Running Evaluation *****
  Num examples = 694
  Batch size = 8
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 694
  Batch size = 8
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 694
  Batch size = 8
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 694
  Batch size = 8
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to 

TrainOutput(global_step=2000, training_loss=0.26685662698745727, metrics={'train_runtime': 808.3637, 'train_samples_per_second': 20.582, 'train_steps_per_second': 2.576, 'total_flos': 952883409202800.0, 'train_loss': 0.26685662698745727, 'epoch': 5.76})

In [9]:
###########Now testing our test file aginst the pretrained model############
###########We will be repeating the same steps we did for train data for our test data as well############
###########Loading Test file#############

test_data = pd.read_csv("FinalTest.csv")
test_data.loc[(test_data['sarcastic'] == 'sarcastic'), 'sarcastic'] = 1
test_data.loc[(test_data['sarcastic'] == 'not_sarcastic'), 'sarcastic'] = 0
X_test = list(test_data["tweet"])
Y_test = list(test_data["sarcastic"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)


In [10]:
test_data

,Id,tweet,sarcastic,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1081209061251710976,so nice not having to worry about finding a st...,1,NaN,NaN,NaN,NaN
1,1043020074670415872,I do love how #banking works- As a customer yo...,1,NaN,NaN,NaN,NaN
2,1126482905688100864,Sometimes I listen to the uga fight song and c...,1,NaN,NaN,NaN,NaN
3,1166817091489542144,"Our magical cup run has come to a halt, it was...",1,NaN,NaN,NaN,NaN
4,1151161501283209222,I’m pretty sure the first four hours back in m...,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
216,1169035797774962688,"I wonder if this will have a chilling effect, ...",0,NaN,NaN,NaN,NaN
217,240984729288065024,school was mentally exhausting today,0,NaN,NaN,NaN,NaN
218,1091698861524037632,dragons-and-bees: thinking about……..her reblog...,0,NaN,NaN,NaN,NaN
219,213343876260433920,"One of the best skins quotes, ""You are a glori...",0,NaN,NaN,NaN,NaN


In [11]:
#########Check the data###############
print("Total Length of the Test.csv File received:",len(X_test))
print("Sarcastic Tweets out of 221 is:",test_data.sarcastic.sum())
print("Non-sarcastic", len(X_test)-test_data.sarcastic.sum())

Total Length of the Test.csv File received: 221
Sarcastic Tweets out of 221 is: 119
Non-sarcastic 102


In [12]:
#######repeating the same step as we did in train to create a torch set###################
test_dataset = Dataset(X_test_tokenized)


In [25]:
###########Here we load the best model that we think was best while training########
###########Loading the checkpoint#############
model_path = "output/checkpoint-2000"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

loading configuration file output/checkpoint-2000/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file output/checkpoint-2000/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceC

In [26]:
###########Using the trainer function again as we did in the training phase#############
test_trainer = Trainer(model)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [27]:
############Results of predicting###############
raw_pred, label_ids, metr = test_trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 221
  Batch size = 8


In [28]:
raw_pred

array([[-4.28146   ,  4.6001263 ],
       [-4.069339  ,  4.241171  ],
       [ 4.325358  , -5.2882237 ],
       [ 4.0157757 , -3.9507477 ],
       [ 4.0817447 , -4.2968    ],
       [ 3.3938754 , -3.4918084 ],
       [ 2.5581417 , -3.3836915 ],
       [ 4.31066   , -4.330642  ],
       [ 4.4495835 , -4.8086705 ],
       [ 4.5771003 , -5.2508707 ],
       [ 3.229751  , -3.116797  ],
       [ 3.9464278 , -3.8892615 ],
       [-4.2272906 ,  4.5240417 ],
       [ 2.3273735 , -2.327002  ],
       [ 0.84680235, -0.9851649 ],
       [ 3.6121364 , -3.513136  ],
       [ 3.0980737 , -2.8881347 ],
       [ 4.485668  , -4.9463987 ],
       [ 4.136115  , -5.2095475 ],
       [ 4.3053374 , -5.237695  ],
       [ 4.5650473 , -5.1531568 ],
       [-4.2243814 ,  4.5507116 ],
       [ 3.8250504 , -3.6656058 ],
       [ 4.1640615 , -4.3053823 ],
       [ 3.6369576 , -3.4942272 ],
       [-3.1038342 ,  3.0417387 ],
       [ 4.3411856 , -4.5239897 ],
       [-3.8824654 ,  4.0563035 ],
       [-3.794125  ,

In [29]:
#######we now have the ra pred which ahas to be pre-processed with argmax where we already defined###########
y_pred = np.argmax(raw_pred, axis=1)

In [30]:
y_pred

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0])

In [31]:
res = [idx
   for idx, elem in enumerate(y_pred)
   if elem != Y_test[idx]
]

In [32]:
# Result
print("The index positions with mismatched values:\n", res)

The index positions with mismatched values:
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 26, 32, 33, 39, 40, 42, 43, 45, 46, 47, 49, 51, 53, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 74, 76, 80, 81, 89, 90, 91, 92, 95, 96, 99, 100, 101, 102, 104, 105, 106, 107, 113, 115, 117, 118, 128, 142, 155, 161, 162, 181, 183, 184, 186, 189, 196, 200, 203, 216]


In [43]:
for i, tweet in enumerate(test_data.iloc[res].tweet):
  print(res[i], tweet)

2 Sometimes I listen to the uga fight song and cry because I miss redcoats but besides that I’m fine 😂😂
3 Our magical cup run has come to a halt, it was good while it lasted boys #NUFC
4 I’m pretty sure the first four hours back in my other non tennis job has been as long as the two weeks I’ve spent at Wimbledon. ...
5 Need about 54 hours of sleep
6 Because infidelities make everything right.
7 @gitcoingg kappa
8 Just spent God knows how long watching 2 of the Rocky films in order to not have to do an essay - time well spent Ains 🙃🤦
9 Just watched the best play in existence- Shrek: the Musical @ Woodford county HS. 

I'm just sad it's ogre.
10 my sister is cool!
11 A lot of time could be saved if #olympics #boxing officials would release their pre-determined results in advance.
13 @HarvardCentrist Holy shit, I bet you guys are a lot of fun at parties.
14 Oh no! I “accidentally” opened a bottle of wine, and there’s no one here to drink it but me.
15 hooorayyy no sleeeppppp ahhhhh
16 @Th

In [33]:
from sklearn import metrics
print("Accuracy: %.3f" % metrics.accuracy_score(Y_test,y_pred))
cm=metrics.confusion_matrix(Y_test,y_pred)
print("\nConfusion Matrix:\n",cm)

Accuracy: 0.615

Confusion Matrix:
 [[88 14]
 [71 48]]


In [34]:
#how many predictions are right out of sarcastic tweets
print("Recall: %.3f" % metrics.recall_score(Y_test,y_pred))

Recall: 0.403


In [35]:
#How many postives are truly identified
print("Precision: %.3f" % metrics.precision_score(Y_test,y_pred))

Precision: 0.774


In [36]:
#Takes both sarcastic and non-sarcastic contents
print("F1: %.3f" % metrics.f1_score(Y_test,y_pred))

F1: 0.530
